In [45]:
import pandas as pd
import tarfile
import tables
from multiprocessing import Pool
from numba import jit

In [46]:
# usual imports
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
msd_subset_path='./MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')
assert os.path.isdir(msd_subset_path),'wrong path' # sanity check

In [47]:
import hdf5_getters as GETTERS

In [48]:
def strtimedelta(starttime,stoptime):
    return str(datetime.timedelta(seconds=stoptime-starttime))

In [49]:
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

In [50]:
# we can now easily count the number of files in the dataset
print('number of song files:',apply_to_all_files(msd_subset_data_path))

number of song files: 10000


In [51]:
all_artist_names = set()

In [52]:
def open_h5_file_read(h5filename):
    """
    Open an existing H5 in read mode.
    Same function as in hdf5_utils, here so we avoid one import
    """
    return tables.open_file(h5filename, mode='r')

In [53]:
# we define the function to apply to all files
def func_to_get_artist_name(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = open_h5_file_read(filename)
    artist_name = GETTERS.get_artist_name(h5)
    all_artist_names.add( artist_name )
    h5.close()

In [54]:
t1 = time.time()
apply_to_all_files(msd_subset_data_path,func=func_to_get_artist_name)
t2 = time.time()
print('all artist names extracted in:',strtimedelta(t1,t2))

all artist names extracted in: 0:01:01.339869


In [55]:
# let's see some of the content of 'all_artist_names'
print('found',len(all_artist_names),'unique artist names')
for k in range(5):
    print(list(all_artist_names)[k])

found 4412 unique artist names
b'Nick Jonas & The Administration'
b'OutKast featuring George Clinton'
b'40 Grit'
b'Catherine Howe & Vo Fletcher'
b'Deodato'


In [56]:
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))

In [57]:
#SQL query
q = "SELECT DISTINCT artist_name FROM songs"
t1 = time.time()
res = conn.execute(q)
all_artist_names_sqlite = res.fetchall()
t2 = time.time()
print('all artist names extracted (SQLite) in:',strtimedelta(t1,t2))
conn.close()
for k in range(5):
    print(all_artist_names_sqlite[k][0])

all artist names extracted (SQLite) in: 0:00:00.004750
!!!
(hed) p.e.
089 Clique feat. Minnesota Snipe & Skinny Cueball
089 Clique feat. Prophet
1. Futurologischer Congress


In [90]:
# Most common artist ID (Not name because name can have different spellings or only be a featuring artist)
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id FROM songs"
res = conn.execute(q)
all_artist_ids = map(lambda x: x[0], res.fetchall())
conn.close()

In [91]:
# The Echo Nest artist id look like:
artistIDS = list(all_artist_ids)

In [92]:
artistIDS

['AR009211187B989185',
 'AR00A6H1187FB5402A',
 'AR00LNI1187FB444A5',
 'AR00MBZ1187B9B5DB1',
 'AR01IP11187B9AF5D2',
 'AR01VU31187B997DA0',
 'AR01W2D1187FB5912F',
 'AR022JO1187B99587B',
 'AR02DB61187B9A0B5E',
 'AR02IU11187FB513F2',
 'AR02KZG1187FB3B3B4',
 'AR02PUA1187FB52574',
 'AR02YGA1187B9B8AC4',
 'AR035N21187FB3938E',
 'AR039B11187B9B30D0',
 'AR03BDP1187FB5B324',
 'AR03GWG1187B9B6C85',
 'AR040M31187B98CA41',
 'AR040QX1187FB4CFE1',
 'AR048JZ1187B9AEB85',
 'AR04KTB1187B99B6B7',
 'AR050VJ1187B9B13A7',
 'AR051KA1187B98B2FF',
 'AR0569B1187B9A4036',
 'AR059HI1187B9A14D7',
 'AR05IU31187B9B9A1A',
 'AR05KQA1187B9963B3',
 'AR05NQH1187B98E875',
 'AR05OJD1187B9B99A6',
 'AR05VW21187FB407B4',
 'AR05XFO1187FB4DAE5',
 'AR061J011A348F060F',
 'AR0693R1187FB59D32',
 'AR06EB01187FB40150',
 'AR06EHB1187FB42914',
 'AR06IFV1187B9977EC',
 'AR06M3F1187FB38BED',
 'AR06USW1187FB4EACA',
 'AR07CRG1187FB594FA',
 'AR07HWO1187B9A2189',
 'AR07QYZ1187FB5BF78',
 'AR07VS41187FB37944',
 'AR08DYV1187B99A0D7',
 'AR08GQF11

In [94]:
# Creating artist dictionary for number of songs by each artsist
files_per_artist = {}
for aid in artistIDS:
    files_per_artist[aid] = 0

In [95]:
def func_to_count_artist_id(filename):
    """
    This function does 3 simple things:
    - open the song file
    - get artist ID and put it
    - close the file
    """
    h5 = open_h5_file_read(filename)
    artist_id = GETTERS.get_artist_id(h5)
    files_per_artist[artist_id.decode("utf-8")] += 1
    h5.close()

In [96]:
# we apply this function to all files
apply_to_all_files(msd_subset_data_path,func=func_to_count_artist_id)

10000

In [97]:
files_per_artist

{'AR009211187B989185': 2,
 'AR00A6H1187FB5402A': 4,
 'AR00LNI1187FB444A5': 2,
 'AR00MBZ1187B9B5DB1': 3,
 'AR01IP11187B9AF5D2': 4,
 'AR01VU31187B997DA0': 1,
 'AR01W2D1187FB5912F': 1,
 'AR022JO1187B99587B': 1,
 'AR02DB61187B9A0B5E': 1,
 'AR02IU11187FB513F2': 6,
 'AR02KZG1187FB3B3B4': 1,
 'AR02PUA1187FB52574': 3,
 'AR02YGA1187B9B8AC4': 5,
 'AR035N21187FB3938E': 7,
 'AR039B11187B9B30D0': 6,
 'AR03BDP1187FB5B324': 10,
 'AR03GWG1187B9B6C85': 1,
 'AR040M31187B98CA41': 2,
 'AR040QX1187FB4CFE1': 2,
 'AR048JZ1187B9AEB85': 1,
 'AR04KTB1187B99B6B7': 1,
 'AR050VJ1187B9B13A7': 3,
 'AR051KA1187B98B2FF': 10,
 'AR0569B1187B9A4036': 3,
 'AR059HI1187B9A14D7': 6,
 'AR05IU31187B9B9A1A': 3,
 'AR05KQA1187B9963B3': 1,
 'AR05NQH1187B98E875': 3,
 'AR05OJD1187B9B99A6': 2,
 'AR05VW21187FB407B4': 3,
 'AR05XFO1187FB4DAE5': 1,
 'AR061J011A348F060F': 1,
 'AR0693R1187FB59D32': 3,
 'AR06EB01187FB40150': 5,
 'AR06EHB1187FB42914': 1,
 'AR06IFV1187B9977EC': 1,
 'AR06M3F1187FB38BED': 1,
 'AR06USW1187FB4EACA': 3,
 'AR07CRG1

In [98]:
# the most popular artist (with the most songs) is:
most_pop_aid = sorted(files_per_artist,
                      key=files_per_artist.__getitem__,
                      reverse=True)[0]
print(most_pop_aid,'has',files_per_artist[most_pop_aid],'songs.')

AROIHOI122988FEB8E has 13 songs.


In [99]:
# of course, it is more fun to have the name(s) of this artist
# let's get it using SQLite
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_name FROM songs"
q += " WHERE artist_id='"+most_pop_aid+"'"
res = conn.execute(q)
pop_artist_names = map(lambda x: x[0], res.fetchall())
conn.close()
print('SQL query:',q)
print('name(s) of the most popular artist:',pop_artist_names)

SQL query: SELECT DISTINCT artist_name FROM songs WHERE artist_id='AROIHOI122988FEB8E'
name(s) of the most popular artist: <map object at 0x7f59f0439e10>


In [100]:
# let's redo all this work in SQLite in a few seconds
t1 = time.time()
conn = sqlite3.connect(os.path.join(msd_subset_addf_path,
                                    'subset_track_metadata.db'))
q = "SELECT DISTINCT artist_id,artist_name,Count(track_id) FROM songs"
q += " GROUP BY artist_id"
res = conn.execute(q)
pop_artists = res.fetchall()
conn.close()
t2 = time.time()
print('found most popular artist in',strtimedelta(t1,t2))
popular = sorted(pop_artists,key=lambda x:x[2],reverse=True)

found most popular artist in 0:00:00.017333


In [103]:
popular

[('AROIHOI122988FEB8E', 'Mario Rosenstock', 13),
 ('AR12F2S1187FB56EEF', 'Aerosmith', 12),
 ('AR9W3X91187FB3994C', 'Phil Collins', 12),
 ('AREWQSE1187B9AEC6C', 'The Jackson Southernaires', 12),
 ('ARIRD6J1187FB5A98C', 'Sugar Minott', 12),
 ('ARJIE2Y1187B994AB7', 'Line Renaud', 12),
 ('ARVN9FZ1187FB393F1', 'Snow Patrol', 12),
 ('ARX9YIP1187B98A656', 'RUN-DMC', 12),
 ('AR6PJ8R1187FB5AD70', 'Shakira', 11),
 ('AR78ZID1187B9B31ED', 'Mikey Dread', 11),
 ('ARH6W4X1187B99274F', 'Radiohead', 11),
 ('ARODBRG1187FB3FD99', 'Nick Cave & The Bad Seeds', 11),
 ('ART3O5Z1187B9AB043', 'Stevie Ray Vaughan And Double Trouble', 11),
 ('AR03BDP1187FB5B324', 'Britney Spears feat. Pharrell Williams', 10),
 ('AR051KA1187B98B2FF', 'Wilks', 10),
 ('AR7KA5V1187FB44E6B', 'The Del McCoury Band', 10),
 ('AR8BMEQ1187B9B4214', 'Pumpkin Buzzard', 10),
 ('ARCCRTI11F4C845308', 'White Heart', 10),
 ('ARDVZTE1187FB5A0A1', 'Bon Jovi', 10),
 ('ARE8GLF1187FB52532', 'Red Hot Chili Peppers', 10),
 ('ARI648V1187B9B5379', 'Micha

In [104]:
list_of_pop = []
for i in popular:
    popular_list = list(i)
    popular_list.remove(popular_list[0])
    list_of_pop.append(popular_list)

In [105]:
popu = pd.DataFrame(list_of_pop,columns=['Artist','# of Songs'])

In [106]:
popu.set_index('Artist', inplace=True)

In [108]:
popu.to_csv('./csv_subset/popular_artists.csv')